In [1]:
import requests
import json

In [2]:
riot_api_key_file = "../../riot_api_key.json"
API_KEY = ""
with open(riot_api_key_file) as f:
    API_KEY = json.loads(f.readlines()[0])["API_KEY"]
    
print(f"Key reading was {'not ' if (API_KEY == '') else ''}successful.")

Key reading was successful.


In [16]:
def runGetSubdomain(in_subdomain_with_params, in_region, in_additional_params_str = ''):
    get_url = f"https://{in_region}.api.riotgames.com/lol/{in_subdomain_with_params}?api_key={API_KEY}{in_additional_params_str}"
    return requests.get(get_url)
    
def runGetSummonerByName(in_summoner_name, in_region):
    subdomain_with_params = f"summoner/v4/summoners/by-name/{in_summoner_name}"
    return runGetSubdomain(subdomain_with_params, in_region)

def runGetMatchListByAccount(in_encrypted_account_id, in_region):
    subdomain_with_params = f"match/v4/matchlists/by-account/{in_encrypted_account_id}"
    return runGetSubdomain(subdomain_with_params, in_region)

def runGetMatchInfoByMatchId(in_match_id, in_region):
    subdomain_with_params = f"match/v4/matches/{in_match_id}"
    return runGetSubdomain(subdomain_with_params, in_region)
    
def runGetMatchTimelineDataByGameID(in_match_id, in_region):
    subdomain_with_params = f"match/v4/timelines/by-match/{in_match_id}"
    return runGetSubdomain(subdomain_with_params, in_region)

def runGetChampionMasteriesBySummonerId(in_encrypted_summoner_id, in_region):
    subdomain_with_params = f"champion-mastery/v4/champion-masteries/by-summoner/{in_encrypted_summoner_id}"
    return runGetSubdomain(subdomain_with_params, in_region)


In [17]:
in_sumName = 'deamonpog'
in_region = 'EUW'

get_response = runGetSummonerByName('deamonpog','euw1')
if get_response.status_code == 200:
    summoner_info = get_response.json()
    print(summoner_info)
else:
    print(f"Response code: {get_response.status_code}")
    
get_response = runGetMatchListByAccount(summoner_info['accountId'],'euw1')
if get_response.status_code == 200:
    match_list = get_response.json()
    print(f"Number of matches: {len(match_list['matches'])}")
else:
    print(f"Response code: {get_response.status_code}")
          
get_response = runGetMatchInfoByMatchId(match_list['matches'][0]['gameId'],'euw1')
if get_response.status_code == 200:
    match_info = get_response.json()
    print(f"The Game mode: {match_info['gameMode']}")
else:
    print(f"Response code: {get_response.status_code}")

get_response = runGetMatchTimelineDataByGameID(match_list['matches'][0]['gameId'],'euw1')
if get_response.status_code == 200:
    match_data = get_response.json()
    print(f"Number of frames: {len(match_data['frames'])}")
else:
    print(f"Response code: {get_response.status_code}")

{'id': 'uy0D0lyCAvNnR71LW5x-0SA7wSd_2-X-IPW_4ItrQGQP3m8', 'accountId': 'gaNAOocsfkvwlC0jYXEySSWKG8930SWqHkJNHq5almX4WLQ', 'puuid': 'olnw7mrmASkBqlCVg4vOzbYnFE4btBrQvmbD_px2AkY5mepTNchqaXDWKbW3IsHXIFuCt42V4tqymA', 'name': 'deamonpog', 'profileIconId': 22, 'revisionDate': 1597442298000, 'summonerLevel': 342}
Number of matches: 100
The Game mode: ARAM
Number of frames: 17


In [13]:
match_data['frames'][4]['events']

[{'type': 'ITEM_DESTROYED',
  'timestamp': 182987,
  'participantId': 4,
  'itemId': 2003},
 {'type': 'WARD_KILL',
  'timestamp': 184671,
  'wardType': 'UNDEFINED',
  'killerId': 10},
 {'type': 'CHAMPION_KILL',
  'timestamp': 188570,
  'position': {'x': 6006, 'y': 5845},
  'killerId': 1,
  'victimId': 7,
  'assistingParticipantIds': [2, 3, 4, 5]},
 {'type': 'ITEM_PURCHASED',
  'timestamp': 192636,
  'participantId': 7,
  'itemId': 3044},
 {'type': 'CHAMPION_KILL',
  'timestamp': 193628,
  'position': {'x': 6618, 'y': 6642},
  'killerId': 2,
  'victimId': 9,
  'assistingParticipantIds': [1, 3, 5]},
 {'type': 'ITEM_DESTROYED',
  'timestamp': 194222,
  'participantId': 5,
  'itemId': 2003},
 {'type': 'CHAMPION_KILL',
  'timestamp': 197363,
  'position': {'x': 7085, 'y': 7348},
  'killerId': 2,
  'victimId': 6,
  'assistingParticipantIds': [1, 3, 4, 5]},
 {'type': 'ITEM_PURCHASED',
  'timestamp': 198882,
  'participantId': 9,
  'itemId': 3010},
 {'type': 'SKILL_LEVEL_UP',
  'timestamp': 19

In [14]:
summoner_info

{'id': 'uy0D0lyCAvNnR71LW5x-0SA7wSd_2-X-IPW_4ItrQGQP3m8',
 'accountId': 'gaNAOocsfkvwlC0jYXEySSWKG8930SWqHkJNHq5almX4WLQ',
 'puuid': 'olnw7mrmASkBqlCVg4vOzbYnFE4btBrQvmbD_px2AkY5mepTNchqaXDWKbW3IsHXIFuCt42V4tqymA',
 'name': 'deamonpog',
 'profileIconId': 22,
 'revisionDate': 1597442298000,
 'summonerLevel': 342}

In [28]:
match_info['participantIdentities'][7]

{'participantId': 8,
 'player': {'platformId': 'OC1',
  'accountId': 'R3GRsIK7VObJc74v2kpmM_DLcXoLRpV3H3ly575eCgO1giE',
  'summonerName': 'deamonpog',
  'summonerId': 'uy0D0lyCAvNnR71LW5x-0SA7wSd_2-X-IPW_4ItrQGQP3m8',
  'currentPlatformId': 'EUW1',
  'currentAccountId': 'gaNAOocsfkvwlC0jYXEySSWKG8930SWqHkJNHq5almX4WLQ',
  'matchHistoryUri': '/v1/stats/player_history/OC1/201473820',
  'profileIcon': 22}}